In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re


In [188]:
#目的：將台北市的所有古蹟放到goole map 自製地圖上
#作法：先將wiki上的古蹟列表擷取下來，再選取在臺北市的古蹟並存入excel中，最後再google自製地圖中上傳xlxs檔


url='https://zh.wikipedia.org/wiki/%E8%87%BA%E7%81%A3%E5%8F%A4%E8%B9%9F%E5%88%97%E8%A1%A8'
tables=pd.read_html(url)

r=requests.get(url)
bs=BeautifulSoup(r.text,'lxml')
#從wiki的台灣古蹟列表中找出所有古蹟，並加上古蹟的等級
his=[]
for i in range(3,8):
    h=i-3
    first_class=tables[i]
    n=bs.find(class_='mw-parser-output').find_all('h3')[h].find(class_='mw-headline').text
    first_class[5]=n
    first_class[5][0]='古蹟等級'
    if i>3: 
        first_class=first_class.drop([0])
    his.append(first_class)
n=his[0].append(his[1]).append(his[2]).append(his[3]).append(his[4])


#加上古蹟所在縣市
city=[]
for i in n[2]:
    citys=i[0:3].replace('臺','台')
    city.append(citys)
n[6]=city
n[6]=n[6].replace('地址','縣市') 

#加上古蹟介紹網址
urls_link=['網址']
for i in range(3,8):
    tables=bs.find_all('table')[i].find_all('tr')[1:]
    for table in tables:
        title=table.find_all('td')[1].find('a')
        if not title==None:   
            url_link=table.find_all('td')[1].a['href']

            urls_link.append('https://zh.wikipedia.org'+url_link)
        else:
            urls_link.append('無資料')

n[7]=urls_link
#print(n)
            
#存入excel
writer=pd.ExcelWriter('/Users/louis/Desktop/taipei.xlsx')
n.to_excel(writer, sheet_name='sheet_1')
writer.save()

     0                   1                2   3     4                  5    6  \
0    #                古蹟名稱               地址  類型  建成年代               古蹟等級   縣市   
1    1  台北府城（北門、東門、南門、小南門）           臺北市中正區  城郭  1884  1997年4月以前公告為一級古蹟者  台北市   
2    2                 紅毛城   新北市淡水區中正路28巷1號  衙署  1629  1997年4月以前公告為一級古蹟者  新北市   
3    3       基隆二沙灣砲臺（海門天險）           基隆市中正區  關塞  1840  1997年4月以前公告為一級古蹟者  基隆市   
4    4               金廣福公館      新竹縣北埔鄉中正路6號  宅第  1835  1997年4月以前公告為一級古蹟者  新竹縣   
..  ..                 ...              ...  ..   ...                ...  ...   
56  57          大同之家（含網球場）       台北市中正區重慶南路   -   NaN   1997年4月以後公告的市定古蹟  台北市   
57  58              西本願寺鐘樓      台北市萬華區中華路一段   -  1923   1997年4月以後公告的市定古蹟  台北市   
58  59            西本願寺樹心會館      台北市萬華區中華路一段   -  1923   1997年4月以後公告的市定古蹟  台北市   
59  60                 晉德宮     台北市萬華區康定路13號  祠廟  1862   1997年4月以後公告的市定古蹟  台北市   
60  61           原臺灣軍司令官官邸  台北市中正區南昌路一段136號  宅第  1909   1997年4月以後公告的市定古蹟  台北市   

                           